이번에는 LCEL를 이용해서 Map Reduce Chain을 직접 구현해 보자.

먼저 단순화한 버전을 만들어본다.

복잡한 기능은 나중에 추가.

Map Reduce 는 어떻게 동작할까?

1. 일단 document의 list를 얻어야 한다.

   ```python
   list of docs
   ```

2. 그 다음으로 list 내부의 모든 document들을 위한 prompt를 만들어준다.

   ```python
   list of docs

   for doc in list of docs | prompt
   ```

3. 그리고 그 prompt는 LLM에게 전달할건데, 기본적인 내용은 다음과 같다.

   '이 document를 읽고, 사용자의 질문에 답변하기에 적절한 정보가 있는지 확인해주세요.'

   ```python
   list of docs

   for doc in list of docs | prompt | llm
   ```

4. 이를 전달받은 LLM은 응답(response)을 출력할거다.
   그리고 llm으로부터 받은 response들을 취합해 하나의 document를 만들어 낸다.

   ```python
   list of docs
   for doc in list of docs | prompt | llm

   for response in list of llms response | put them all together
   ```

5. 이렇게 만들어진 단 하나의 최종 document가, llm을 위한 prompt로 전달된다.

   ```python
   list of docs
   for doc in list of docs | prompt | llm
   for response in list of llms response | put them all together

   final doc | prompt | llm
   ```

그러면 마침내 처음의 질문에 대한 답변이 생성이 될 것이다.


처음부터 정리해보면, 우리는 '빅토리 맨션을 묘사해주세요' 라는 질문을 할 건데

```python
   list of docs

   for doc in list of docs | prompt | llm

   for response in list of llms response | put them all together

   final doc | prompt | llm

   chain.invoke("Describe Victory Mansions")
```

```python
그 질문은 먼저 retriever에게 전달될 거다.

그럼 retriever는 빅토리 맨션을 묘사하는것과 관련이 있는 document list를 반환한다.                    - list of docs

그 list의 모든 document 에 대한 prompt 를 만들고, 그걸 LLM에 전달한다.                              - for doc in list of docs | prompt | llm

이런 내용을 담고 있을거다 '이 문서를 읽고, 질문에 답하는데에 관련이 있는 중요한 정보를 추출하세요.'

그 작업이 list의 모든 document에 수행된다.

만약 5개의 document가 있다면, LLM에게 5번 질문해서 5개의 응답(response)를 받는거다.                  - for response in list of llms response | put them all together

그리고 이 응답들을 전부 묶어서 하나의 긴 document를 만든다.

그 최종 document 하나(final doc)가 prompt에 입력되어 LLM에게 전달되게 된다.                         - final doc | prompt | llm

promtp의 내용은 '이것은 질문과 관련이 있는 정보들입니다. 이를 사용하여 대답해주세요'와 같다.
```

---


```python
   list of docs

   for doc in list of docs | prompt | llm

   for response in list of llms response | put them all together

   final doc | prompt | llm

   chain.invoke("Describe Victory Mansions")
```

LangChain Express Language를 사용해서 위 로직을 구현해보자.

끝 부분부터 작성해본다. chain

먼저 chain의 마지막 구성요소는 LLM 이다.

그리고 그 앞에 llm에게 전달될 최종 final prompt가 필요하다.

final_prompt에는 context, question이 들어갈 텐데,

context에는 많은 document들의 발췌문들(extracted parts)이 삽입될 거다.

그리고 question에는 사용자가 입력할 질문이 올거고.


In [ ]:
from langchain.prompts import ChatPromptTemplate

# list of docs

# for doc in list of docs | prompt | llm

# for response in list of llms response | put them all together

# final doc | prompt | llm : chain의 마지막 구성요소는 LLM이다.
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a 
            question, create a final answer.
            If you don't know the answer, just say that you don't know.
            Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = final_prompt | llm

chain.invoke("Describe Victory Mansions")

이제부터 template이 필요로하는 이 context를 만드는 방법을 찾아내야 한다.

이 context에 입력되는 긴 document는 LLM이 추출한 여러 다른 document들의 작은 부분들을 모아서 만들어질 거다.

먼저 ("human", "{question}"), 부분을 수정하는 것부터 해보자.

우리는 질문을 template의 입력으로 전달하는 방법을 알고있다.

```python
    final_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                Given the following extracted
                .
                .
                ...
                ------
                {context}
                """,
            ),
            ("human", "{question}"),
        ]
    )

    chain = {"question": RunnablePassthrough()} | final_prompt | llm

    chain.invoke("Describe Victory Mansions")
```

그럼 아래 질문이 final_prompt의 question에 전달되게 된다.


이제 이 context를 만드는 방법을 생각해보자.

chain 부분에 context property를 만들고 또 다른 chain을 생성해준다.

```python
    chain = {"context": ?, "question": RunnablePassthrough()} | final_prompt | llm
```

? 부분에 생성한 chain을 넣어준다. 우리는 map_chain이라 해보자.

그럼 map_chain은 무슨일을 해야 할까?

```python
    chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm
```

우선 map_chain은 마지막 chain 내부에서 호출하게 될거다.

따라서 invoke 메서드가 실행될 때, map_chain이 context로 들어갈 거고,

그리고 RunnablePassthrough() 에는 질문("Describe Victory Mansions")이 들어갈 것이다.

그리고 실행 결과는 모두 final_prompt에 전달된다.


이제 map_chain에 대해 알아보자.

먼저 map chain에는 document가 필요하다.

document는 retriever를 사용해서 얻을 수 있다.

또 사용자의 질문 내용을 알아야 한다.

그래야 LLM에게 각 document를 살펴보면서 사용자 질문에 대답하는데 필요한 정보가 담겨있는지 봐달라고 요청할 수 있다.

그럼 이제 두 개의 데이터가 필요하다는 것을 알았다. 'document 와 사용자 질문'

먼저 document부터 해보자.

우리는 retriever로부터 document를 받아올거다.

```python
    map_chain = { "documents": retriever }
```

일단 chain.invoke 메서드를 호출하면, 입력되어있는 질문(text)가 동작을 한다.

map_chain에 입력되어서 내부에서 retriever의 실행에 사용되고, 그 결과로 많은 document가 출력될거다. document의 list 반환

그리고 사용자가 보낸 질문(question)도 전달해야 한다.

왜냐하면 retriever가 우리에게 document를 제공하면, 우리는 어떤 function을 실행할건데,

거기서 document가 사용자의 질문(question)과 관련된 정보를 가졌는지를 확인해야 하기 때문이다.

그래서 map_chain에도 question 값을 넣어줘야 한다.

```python
    map_chain = { "documents": retriever,  "question": RunnablePassthrough()}
```

여기까지가 map_chain 구현 파트 1 이다.

이 코드가 하는 일은 우리에게 이런 형태의 자료를 만들어주는 것이다.

```python
    {
        "documents":[Documents],
        "question": "Describe Victory Mansions",
    }
```

이제 list의 각 document별로 작업을 수행하고 그것들이 question에 대한 답변을 하는 데 필요한 관련 정보를 포함하는지 확인해보자.

이 작업을 위한 함수로 map_docs(map_documents)를 작성해준다.

이 함수는 방금 위에서 작성했던 값을 input으로 입력받을거다.

각 값을 input에서 추출한다.

```python

    def map_docs(inputs):
        documents = inputs['documents']
        question = inputs['question']
```

이 함수는 어떻게 호출할 수 있을까?

이를 위해 RunnableLambda 라는 class가 있다.

RunnableLambda는 chain과 그 내부 어디에서든 함수를 호출할 수 있도록 해준다.

```python
    from langchain.schema.runnable import RunnableLambda

    def map_docs(inputs):
        documents = inputs['documents']
        question = inputs['question']

    map_chain = { "documents": retriever,  "question": RunnablePassthrough() } | RunnableLambda(map_docs)

```

이제 우리는 map_chain을 갖게 되었다.


이제는 map_docs의 반환값(return)에 대해 생각해보자.

우리의 chain은 context 값을 얻기 위해 map_chain을 실행할 거다.

```python
    chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm
```

map_chain은 document와 사용자의 question을 입력받는다.

그리고 map_chain의 목표는, 한 개의 string을 반환하는 것이다.

사용자의 question에 대한 답변 생성에 관련이 있는 정보를 포함한 document의 일부나 전체를 말해

map_docs의 반환값(return)은 한 개의 string이 되어야 한다.

우리가 원하는 것은 inputs의 retriever로부터 반환받은 document의 개별 요소마다 또 다른 chain을 실행하는 것이다. 각각 따로!

또 다른 chain을 통해 관련이 있는 정보를 추출해 낼 거다.

이제 또 다른 chain을 만들어 보자.


남은 기능들

- for doc in list of docs | prompt | llm
- for response in list of llms response | put them all together

```python
    for doc in list of docs | prompt | llm
```

list 속 모든 document를 각각 prompt에 넣어서 llm에게 전달해야 한다.

이를 위해 map_doc_chain 를 선언해주자.

```python
    map_doc_chain = map_doc_prompt | llm
```

먼저 map_doc_prompt 를 만들어보자.

```python
    map_doc_prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question.
            Return any relevant text verbatim.
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ])
```

시스템 부분: <br>
다음의 긴 문서의 일부 중 질문에 대한 답변을 생성하는 것과 관련이 있는 부분을 찾아줘.<br>
관련이 있는 부분을 찾았다면 해당 text를 그대로 반환해줘라.

관련있는 text는 변경하지 말고, 그냥 반환해달라는 의미.

이제 documents 내부의 각 document에 대해 이 chain을 실행해 준다.

그리고 각각의 response(응답)을 저장할 responses라는 list를 만든다.

그리고 그걸 하나의 긴 string으로 변환할 거다. 한 개의 긴 document로.

```python
    map_doc_prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question.
            Return any relevant text verbatim.
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ])

    map_doc_chain = map_doc_prompt | llm

    def map_docs(inputs):
        documents = inputs['documents']
        question = inputs['question']

        for document in documents:
            # result는 AI message이고, .content를 가지고 있다.
            result = map_doc_chain.invoke({
                "context": document.page_content,
                "question": question
            }).content
        return

    map_chain = { "documents": retriever,  "question": RunnablePassthrough() } | RunnableLambda(map_docs)
```

list의 각 document에 대해 map_docs_chain을 호출(invoke) 하고있다.

context로 document.page_content가 입력되었고, 사용자가 입력한 question도 입력되었다.

이제 각 result를 저장해줄 results라는 list를 만들어보자.

그리고 join메서드를 사용해 results의 아이템들을 하나의 string으로 합쳐보자. (줄바꿈을 통해 각각을 구분한다("\n\n"))

```python
    def map_docs(inputs):
        documents = inputs['documents']
        question = inputs['question']

        results = []

        for document in documents:
            # result는 AI message이고, .content를 가지고 있다.
            result = map_doc_chain.invoke({
                "context": document.page_content,
                "question": question
            }).content
            results.append(result)
        results = "\n\n".join(results)
        return results
```


전체 코드


In [1]:
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


# llm
llm = ChatOpenAI(temperature=0.1)
# 캐시 경로
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

# vectorstore
vectorstore = FAISS.from_documents(docs, embeddings)

# retriever
retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
        Use the following portion of a long document to see if any of the text is relevant to answer the question. 
        Return any relevant text verbatim.
        -------
        {context}
        """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    results = []
    for document in documents:
        # result는 AI message이고, .content를 가지고 있다.
        result = map_doc_chain.invoke(
            {
                "context": document.page_content,
                "question": question,
            },
        ).content
        results.append(result)
    results = "\n\n".join(results)
    return results


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a 
            question, create a final answer.
            If you don't know the answer, just say that you don't know.
            Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a dilapidated residential building complex located in London, the chief city of Airstrip One in the province of Oceania. The building is described as having a telescreen in the living room, a tiny kitchen with minimal food supplies, and a small table to the left of the telescreen. It has glass doors that let in gritty dust, a hallway that smells of boiled cabbage and old rag mats, and a colored poster of an enormous face with a heavy black mustache and ruggedly handsome features. The building has a faulty lift, with the electric current cut off during daylight hours. The flat in Victory Mansions is located seven flights up, with a telescreen on the wall that cannot be completely shut off. The world outside the window is cold and colorless, with little eddies of wind whirling dust and torn paper into spirals. Additionally, Victory Mansions is a significant location in London as it provides a view of the four Ministries of the government: the Minist

코드를 개선해보자.


In [2]:
map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
        Use the following portion of a long document to see if any of the text is relevant to answer the question. 
        Return any relevant text verbatim.
        -------
        {context}
        """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    # 즉석으로 list만들기
    return "\n\n".join(map_doc_chain.invoke({
        "context": doc.page_content,
        "question":question
    }).content for doc in documents)


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a 
            question, create a final answer.
            If you don't know the answer, just say that you don't know.
            Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building complex in London, specifically in the chief city of Airstrip One in the province of Oceania. It is a dilapidated residential building with cramped living conditions, lack of privacy, and constant surveillance through telescreens. The building is run-down, with a sense of oppression and control looming over its residents. Inside, there are seven flights of stairs, a malfunctioning elevator due to electricity cuts, and a flat with a telescreen that cannot be turned off, broadcasting figures related to pig-iron production. The exterior of Victory Mansions is adorned with a large colored poster of an enormous face with the caption "BIG BROTHER IS WATCHING YOU." The surrounding landscape includes rotting nineteenth-century houses, bombed sites, and sordid colonies of wooden dwellings, emphasizing the bleak and rundown atmosphere of the area.')

# Recap


저번에는 MapReduce Chain을 만들어서 사용했다.

이번에는 LCEL 를 사용해서 직접 만들어봤다.

MapReduce 의 동작 원리를 되짚어보자.

retriever로부터 documents들을 입력받고, LLM을 통해, 추출된 document를 살펴보면서 

사용자 질문에 대한 답변을 생성하는 것과 관련이 있는 부분을 가지고 있는지 살펴봤다.

그렇게 각 document에 대해 생성된 응답들을 전부 합쳐 한 개의 최종 final document를 만들고,

그 후에 document를 final prompt에 입력하여 LLM에 전달하면 LLM은 질문에 대한 답변을 생성해주었다.

우리가 먼저 한 일은 코드의 마지막 부분부터 작업을 시작한 것.

마지막에 llm이 있었고 그 앞에 final_prompt가 있었다. 이게 llm에 전달됨.

final_prompt에 필요한건 추출된 부분을 모두 합친 긴 document와 사용자 question 이였다.

```python
chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm
chain.invoke("Describe Victory Mansions")
```






그리고 question과 context를 final_prompt에게 전달하기 위해 했던 걸 떠올려 보자.

question의 경우 RunnablePassthrough를 이용해서 간단히 처리했다.

사용자가 아래 입력한 질문이 RunnablePassthrough로 옮겨져서 final_prompt에 question 값으로 전달됨

하지만 context의 경우, 모든 추출된 document들을 합친 최종 context를 얻기 위해 map_chain을 실행해야 했다.

map_chain은 마지막 부분에 RunnableLambda를 갖고 있는데, 입력된 아무 함수를 실행할 수 있게 해준다. 우리는 map_docs 함수를 입력해줬다.

map_docs 함수는 retriever로부터 얻은 document들과 사용자의 question을 필요로 했다.
```python
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    # 즉석으로 list만들기
    return "\n\n".join(map_doc_chain.invoke({
        "context": doc.page_content,
        "question":question
    }).content for doc in documents)


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

```

question은 또 RunnablePassthrough를 사용해 쉽게 전달했다.

documents의 값으로는 retriever를 넣어주었다.

retriever는 LangChain에 의해 자동적으로 사용자의 입력값을 받아 호출되는데,

retriever는 document들을 반환해준다.

즉, retriever는 document들을 반환하고, question에는 사용자의 질문이 들어있는 것.

우리는 
```python
{
    "documents": retriever,
    "question": RunnablePassthrough(),
}
``` 
을 map_docs 함수의 입력으로 전달해 주었다.

map_docs 함수에서는 일단 inputs으로 받은 것들을
```python
documents = inputs["documents"]
question = inputs["question"]
```

이렇게 풀어놓았다.

그 다음엔 list 내부의 모든 document에 대해 또다른 chain을 실행했다. 
```python
map_doc_chain.invoke({
        "context": doc.page_content,
        "question":question
    }).content for doc in documents
```



이 chain은 map_doc_prompt 로 부터 시작되었다.

document를 읽고, 관련 정보가 있다면 해당 부분을 반환해달라는 내용을 담고 있다.

```python
map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
        Use the following portion of a long document to see if any of the text is relevant to answer the question. 
        Return any relevant text verbatim.
        -------
        {context}
        """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm
```

map_doc_chain 은 이 prompt 를 llm 에게 전달한다.

여기서는  map_doc_chain.invoke()
```python
map_doc_chain.invoke({
        "context": doc.page_content,
        "question":question
    }).content for doc in documents
```

retriever로부터 반환받은 모든 document에 대해 map_doc_chain을 호출했고,

답변은 항상 AI Message이기 때문에, content부분을 반환하도록 했다.

그리고 chain의 모든 호출(invoke)마다 doc.page_content와 question이 입력되었다.
